# Actividad Guiada 3 de Algoritmos de Optimizacion

Nombre: Roberto Saul Cova Rocamora <br>
[enlace google colab] <br>
[enlace github]

In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)


"gzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())


In [4]:
#Distancia entre nodos
problem.get_weight(0, 1)


15

In [5]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion 

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [6]:
solucion = crear_solucion(Nodos)
print(solucion)
distancia_total(solucion, problem)

[0, 32, 14, 37, 28, 15, 11, 40, 27, 23, 8, 16, 13, 35, 4, 12, 39, 24, 2, 5, 26, 10, 21, 18, 31, 6, 34, 20, 36, 3, 25, 29, 17, 41, 1, 7, 38, 33, 19, 9, 30, 22]


4491

## Busqueda Aleatoria

In [7]:
def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion 

In [8]:
    
#Busqueda aleatoria con N iteraciones    
solucion = busqueda_aleatoria(problem, 5000)

Mejor solución: [0, 20, 34, 27, 1, 28, 32, 19, 23, 38, 36, 31, 22, 18, 24, 3, 26, 41, 39, 21, 2, 10, 16, 33, 35, 14, 15, 13, 17, 37, 6, 7, 11, 12, 4, 29, 30, 40, 9, 25, 8, 5]
Distancia     : 3766


## Busqueda Local

In [9]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = solucion
  mejor_distancia = distancia_total(solucion, problem)
  
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      
      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion



In [10]:
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 
nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3766
Distancia Mejor Solucion Local: 3459


In [11]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(True):
    iteracion +=1         #Incrementamos el contador

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion,mejor_distancia
      
    solucion_referencia = vecina

In [12]:
sol,_ = busqueda_local(problem)

En la iteracion  38 , la mejor solución encontrada es: [0, 1, 6, 26, 18, 10, 41, 23, 40, 9, 2, 27, 30, 38, 22, 24, 21, 39, 29, 4, 5, 13, 19, 37, 17, 31, 32, 28, 8, 25, 11, 12, 3, 34, 33, 20, 35, 36, 15, 16, 14, 7]
Distancia     : 1865


In [13]:
def multi_arranque(N=10):
    mejor_dist = float('inf')
    mejor_sol = []
    for i in range(N):
        sol,dist = busqueda_local(problem)
        if dist < mejor_dist:
            mejor_dist = dist
            mejor_sol = sol

    return mejor_sol,mejor_dist

In [14]:
sol, dist = multi_arranque(5)
print(dist)

En la iteracion  30 , la mejor solución encontrada es: [0, 1, 6, 5, 4, 29, 39, 22, 38, 34, 33, 20, 17, 7, 37, 36, 35, 31, 32, 30, 28, 27, 3, 2, 9, 40, 24, 21, 14, 15, 16, 19, 13, 26, 10, 8, 23, 41, 25, 11, 12, 18]
Distancia     : 1885
En la iteracion  35 , la mejor solución encontrada es: [0, 31, 17, 36, 35, 20, 33, 34, 32, 7, 37, 15, 16, 14, 11, 25, 41, 23, 9, 2, 27, 28, 30, 21, 40, 24, 39, 29, 5, 13, 19, 1, 3, 4, 6, 26, 18, 12, 10, 8, 22, 38]
Distancia     : 1851
En la iteracion  32 , la mejor solución encontrada es: [0, 30, 38, 22, 24, 40, 21, 39, 29, 28, 14, 15, 16, 19, 13, 5, 6, 1, 7, 17, 31, 32, 27, 2, 3, 4, 18, 12, 11, 25, 10, 8, 34, 33, 20, 35, 36, 37, 26, 41, 23, 9]
Distancia     : 1872
En la iteracion  36 , la mejor solución encontrada es: [0, 32, 34, 33, 20, 1, 6, 26, 4, 3, 2, 27, 17, 31, 35, 36, 37, 5, 41, 23, 40, 24, 21, 39, 22, 9, 8, 10, 25, 11, 12, 18, 13, 19, 14, 16, 15, 7, 28, 29, 38, 30]
Distancia     : 1777
En la iteracion  36 , la mejor solución encontrada es: [0, 2

Aplicamos busqueda en entornos variables, el cambio será proporcional a la distancia

In [15]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = solucion
  mejor_distancia = distancia_total(solucion, problem)
  cambio_factor = 1 if mejor_distancia//1000 < 1 else mejor_distancia//1000 # Aplicamos un factor de cambio de posiciones

  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + solucion[j:j+cambio_factor] + solucion[i+1:j] + [solucion[i]] + solucion[j+cambio_factor:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [16]:
#random.seed(42)
solucion = crear_solucion(Nodos)
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 
nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 4940
Distancia Mejor Solucion Local: 4497


In [17]:
#random.seed(42)
sol,_ = busqueda_local(problem)

En la iteracion  35 , la mejor solución encontrada es: [0, 3, 32, 34, 33, 20, 35, 36, 31, 17, 37, 15, 7, 1, 5, 26, 18, 25, 11, 12, 13, 19, 16, 14, 6, 4, 2, 28, 30, 38, 22, 8, 10, 41, 23, 40, 24, 21, 39, 9, 29, 27]
Distancia     : 1591


## Simulated Annealing (Enfriamiento Simulado)

In [200]:
#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):
  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]


#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

# Genera 1 solucion usando un nodo aleatorio e insertando al lado otro nodo aleatorio
# entre sus 10 nodos más cercano 
def genera_vecina_aleatorio2(solucion):
  i = random.randint(1,len(solucion)-1)
  distances = []
  for j in range(1,len(solucion)-1):
    if i!=j:
      distances.append((i,j,problem.get_weight(solucion[i],solucion[j])))
  
  distances = sorted(distances,key=lambda x: x[2])

  i,j,_ = random.choice(distances[:20])
  i,j = sorted([i,j])

  return solucion[:i+1] + [solucion[j]] + solucion[i+1:j] + solucion[j+1:]

In [202]:
def recocido_simulado(problem, TEMPERATURA,genera_vecina):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []               #x* del seudocodigo
  mejor_distancia = float('inf')    #F* del seudocodigo
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina = genera_vecina(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
    
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

In [212]:
sol  = recocido_simulado(problem, 10000000,genera_vecina_aleatorio)
sol  = recocido_simulado(problem, 10000000,genera_vecina_aleatorio2)

La mejor solución encontrada es [0, 27, 28, 29, 21, 40, 24, 38, 34, 37, 7, 1, 6, 5, 26, 18, 12, 9, 39, 22, 33, 20, 35, 36, 17, 15, 16, 4, 3, 2, 30, 32, 31, 14, 19, 13, 11, 10, 25, 23, 41, 8]
con una distancia total de 2073
La mejor solución encontrada es [0, 27, 2, 3, 4, 1, 6, 5, 7, 37, 15, 16, 14, 17, 31, 35, 36, 19, 13, 26, 18, 28, 29, 30, 38, 22, 39, 21, 24, 40, 23, 41, 9, 8, 10, 25, 12, 11, 32, 34, 33, 20]
con una distancia total de 1622


La segunda version, que genera soluciones menos aleatorias, suele funcionar mejor que generar dos nodos puramente aleatorios